# Simon's Travel Tracker
### Testing out Folium for Geovisualization


In [265]:
import sys, os, glob

import folium
from geopy.geocoders import Nominatim

sys.path.insert(0, './external_plugins')
import Externals

import pandas

import branca.colormap as cm

locations_dir = './locations/'
visited_cities_list = 'visited-cities.csv' 
nvisited_cities_list = 'want-to-visit.csv'
visited_geos_dir = '{}{}'.format(locations_dir, 'visited/')
nvisited_geos_dir = '{}{}'.format(locations_dir, 'want-to-visit/')

In [256]:
# Get Lat and Long of cities
geolocator = Nominatim(timeout=1000000)

# For cities that I have vistied
visited_lats_longs = []
visited_cities = []

# For cities I want to visit
nvisited_lats_longs = []
nvisited_cities = []

visited_df = pandas.read_csv('{}{}'.format(locations_dir, visited_cities_list), sep=',')
visited_rows = visited_df.shape[0]

nvisited_df = pandas.read_csv('{}{}'.format(locations_dir, nvisited_cities_list), sep=',')
nvisited_rows = nvisited_df.shape[0]

# Add the cities I've visited to a list
print("Visited Cities:")
for x in range(visited_rows):
    d = visited_df.iloc[x]['Visited Cities']
    print(d)
    visited_cities.append(d)

# Add the cities that I want to visit to a list
print("\nCities I Want to Visit")
for x in range(nvisited_rows):
    d = nvisited_df.iloc[x]['Want to Visit']
    print(d)
    nvisited_cities.append(d)

# Get the latitude and logitude of cities I have visited
print("\nCoordinates of cities I have visited")
for city in visited_cities:
    location = geolocator.geocode(city)
    print(location)
    visited_lats_longs.append([location.latitude, location.longitude])

# Get the latitude and logitudes of cities that I want to visit
print("\nCoordinates of cities I want to visit")
for city in nvisited_cities:
    location = geolocator.geocode(city)
    print(location)
    nvisited_lats_longs.append([location.latitude, location.longitude])

Visited Cities:
Houston
San Antonio
Lubbock
Dallas
Galveston
Las Vegas
Los Angeles
Hong Kong
Beijing
Shanghai
Tokyo
Addis Ababa
Dessie
Amsterdam

Cities I Want to Visit
Taipei
Seoul
Jeju
Busan
Kyoto
Okinawa
Stockholm
Berlin
London
Oslo
Brussels
Brisbane
Sydney
Melbourne
Astana
Moscow
Chengdu
Xi\'an
Hue
Ho Chi Minh City
Hanoi
New Delhi
Phnom Penh
Jakarta
Honolulu
Cape Town
Kuala Lumpur
Bangkok
Manila
Lima
Rio de Janeiro
Buenos Aires
Copenhagen
Venice

Coordinates of cities I have visited
Houston, Harris County, Texas, United States of America
San Antonio, Bexar County, Texas, United States of America
Lubbock, Lubbock County, Texas, United States of America
Dallas, Dallas County, Texas, United States of America
Galveston, Galveston County, Texas, United States of America
Las Vegas, Clark County, Nevada, United States of America
LA, Los Angeles County, California, United States of America
香港 Hong Kong, 中西區 Central and Western District, 香港島 Hong Kong Island, HK, PACIFIC PLACE, 中国
北京, 东城区, 

In [257]:
# Sanity check
for result in visited_lats_longs:
    print(result[0], result[1])
    
for result in nvisited_lats_longs:
    print(result[0], result[1])

29.7589382 -95.3676974
29.4246002 -98.4951405
33.5778631 -101.8551665
32.7762719 -96.7968559
29.299328 -94.7945882
36.1662859 -115.149225
34.054935 -118.244476
22.2793278 114.1628131
39.9059631 116.391248
31.2253441 121.4888922
34.6968642 139.4049033
9.0 38.75
11.122604 39.634982
52.3745403 4.89797550561798
25.0375198 121.5636796
37.5666791 126.9782914
33.3940067 126.5626688
35.1799528 129.0752365
35.0231321 135.7634074
26.5707754 128.0255901
59.3251172 18.0710935
52.5170365 13.3888599
51.5073219 -0.1276474
59.9132694 10.7391112
50.8465573 4.351697
-27.4689682 153.0234991
-33.8548157 151.2164539
-37.8142176 144.9631608
51.15092055 71.4388595154859
55.7507178 37.6176606
30.6765553 104.0612783
34.2660171 108.9424252
47.1817585 19.5060937
10.7758439 106.7017555
21.0292095 105.85247
28.6141793 77.2022662
11.568271 104.9224426
-6.1753942 106.827183
21.304547 -157.8556764
-33.928992 18.417396
3.1546872 101.7136362
13.7538929 100.8160803
14.5906216 120.9799696
-12.0621065 -77.0365256
-22.9110

In [269]:
# Note: Need to confirm differente between add_child and add_to

f = folium.Figure(width=1000, height=500) # Instantiate a figure
travel_map = folium.Map(location=[0, 0], zoom_start=2, min_zoom=2).add_to(f) # Instantiate a world map and add it to the figure
travel_map.add_child(folium.features.LatLngPopup()) # Add Latitude Longitude Popup to the map
fs = Externals.Fullscreen()
szt = Externals.ScrollZoomToggler()

fs.add_to(travel_map) # Add a Fullscreen button to the map (plugin)
szt.add_to(travel_map) # Add the scroll zoom toggler button to the map (plugin)

# Colorscale: Steps of 10 between 0 and 200
colorscale = cm.linear.PuRd.scale(0, 200)
colorscale.caption = 'Total number of days spent in each location (Coming Soon)'
travel_map.add_child(colorscale)

"""
Add the .geo.json data for countries to the map first
"""
# Get *.geo.json for visited countries 
for root, dirs, files in os.walk(visited_geos_dir):
    visited_jsons = files
    print(visited_jsons)
    
# Get *.geo.json for countries not visited
for root, dirs, files in os.walk(nvisited_geos_dir):
    nvisited_jsons = files
    print(nvisited_jsons)

# Create feature groups for the countries
visited_map_countries = folium.FeatureGroup(name='Countries I\'ve Visited') 
nvisited_map_countries = folium.FeatureGroup(name='Countries I want to visit')

y = 0 # Indexing for countries that I haven't visited
for x in range(0, len(visited_jsons) + len(nvisited_jsons)):
    if x < len(visited_jsons):  # For visited countries
            map_loc = folium.GeoJson(open('{}{}'.format(visited_geos_dir, visited_jsons[x]), encoding = "utf-8-sig").read(),
                                     style_function=lambda x: {'fillColor': 'green', 'dashArray': '5, 5'},
                                     highlight_function=lambda x: {'weight': 2, 'fillColor': 'blue'}) # Load the .geo.json data and customize
            visited_map_countries.add_child(map_loc) # Add the GeoJSON to the map_countries feature group
            
    else: # For countries not yet visited
            map_loc = folium.GeoJson(open('{}{}'.format(nvisited_geos_dir, nvisited_jsons[y]), encoding = "utf-8-sig").read(),
                                     style_function=lambda x: {'fillColor': 'red', 'dashArray': '5, 5'},
                                     highlight_function=lambda x: {'weight': 2, 'fillColor': 'blue'}) # Load the .geo.json data and customize
            nvisited_map_countries.add_child(map_loc) # Add the GeoJSON to the map_countries feature group
            y+=1

visited_map_countries.add_to(travel_map) # Add the feature group to the map
nvisited_map_countries.add_to(travel_map) # Add the feature group to the map        

"""
Next, add the processed geopy data for cities
"""
visited_map_cities = folium.FeatureGroup(name='Cities I\'ve Visited') 
nvisited_map_cities = folium.FeatureGroup(name='Cities I want to visit')

y = 0 # Indexing for cities that I haven't visited
for x in range(0, len(visited_lats_longs) + len(nvisited_lats_longs)):
    if x < len(visited_lats_longs):  # For visited cities
        mkr = folium.Marker([visited_lats_longs[x][0], 
                             visited_lats_longs[x][1]], 
                            popup=visited_df.iloc[x]['Visited Cities'], 
                            icon=folium.Icon(color='green', prefix='fa', icon='plane')) # Add a marker to show the name of the city located at the marker on click
        visited_map_cities.add_child(mkr) # Add the marker to the feature group
    else: # For cities not yet visited
        mkr = folium.Marker([nvisited_lats_longs[y][0], 
                            nvisited_lats_longs[y][1]], 
                            popup=nvisited_df.iloc[y]['Want to Visit'], 
                            icon=folium.Icon(color='red', prefix='fa', icon='plane')) # Add a marker to show the name of the city located at the marker on click
        nvisited_map_cities.add_child(mkr) # Add the marker to the feature group
        y+=1

visited_map_cities.add_to(travel_map) # Add the feature group to the map
nvisited_map_cities.add_to(travel_map) # Add the feature group to the map  

['CA.geo.json', 'CHN.geo.json', 'ETH.geo.json', 'HK.geo.json', 'JPN.geo.json', 'NLD.geo.json', 'NV.geo.json', 'TX.geo.json']
['ARG.geo.json', 'AUS.geo.json', 'BEL.geo.json', 'BRA.geo.json', 'DEU.geo.json', 'DNK.geo.json', 'FIN.geo.json', 'GBR.geo.json', 'HI.geo.json', 'IDN.geo.json', 'IND.geo.json', 'ITA.geo.json', 'KAZ.geo.json', 'KHM.geo.json', 'KOR.geo.json', 'MYS.geo.json', 'PER.geo.json', 'PHL.geo.json', 'RUS.geo.json', 'SWE.geo.json', 'THA.geo.json', 'TWN.geo.json', 'VNM.geo.json', 'ZAF.geo.json']


In [270]:
folium.LayerControl().add_to(travel_map) # Add Layer Controller to the map

# Add a legend
legend_html = '''
    <div style="position: fixed; 
        bottom: 50px; left: 50px; width: 100px; height: 90px; 
        border:2px solid grey; z-index:9999; font-size:14px;
        ">&nbsp; Legend <br>
          &nbsp; Visited &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
          &nbsp; Will Visit &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>
    </div>
'''
travel_map.get_root().html.add_child(folium.Element(legend_html))   

travel_map # PREVIEW!

In [271]:
# Export 
travel_map.save('index.html')
